# ETL for Non-Text data of Udemy Course Dataset
***
* The raw Udemy course dataset includes (1) a course list of business analytics web crapped from Udemy's business analytics webpages, and (2) detailed information of each course web crapped from course webpage.
* The course information includes non-text data, such as enrollment numbers and ratings, and text data of course description and instruction introduction.  
* This notebook aims to perform the extract, transform, and load (ETL) procedure on the non-text data of the raw dataset. The text data is processed in a seperate notebook. 
***

In [1]:
import pandas as pd
import os.path
import codecs
import re

## 1. Process the Dataset of Course List.

In [2]:
data = pd.read_table("../data/course_list.tsv",encoding='utf-8')
data = data.iloc[:, :-1]
data = data.drop(columns=['link'])
print(data.columns)
print(data.shape)

Index(['id', 'name', 'seller', 'price', 'original_price', 'lectures', 'hours',
       'level'],
      dtype='object')
(1013, 8)


### Handle Missing data

In [3]:
data['lectures'].fillna(value="All Levels", inplace=True)
data["price"].replace("Free","0.0", inplace=True)
data["price"] = data["price"].astype(float)
index = data.index[data["original_price"].isna()]
data.loc[index,"original_price"] = pd.Series(data.loc[index]["price"])
data["level"] = data["level"].astype('category')

In [4]:
total_num = data.shape[0]
miss_course = []
for i in range(total_num):
    if not os.path.exists('../data/raw/'+str(i+1)+'_info.txt'):
        miss_course.append(i)
print(miss_course)
data = data.drop(data.index[miss_course])
# data = data.reset_index(drop=True)

[372, 494, 868, 914]


## 2. Process the Dataset Scrapped from Course Webpage

In [5]:
title = []
enrollment = []
rating = []
num_ratings = []
description = []
five_stars = []
four_stars = []
three_stars = []
two_stars = []
downloadable_resources = []

### Parse variables

In [6]:
for i in range(total_num):
    fname = '../data/raw/'+str(i+1)+'_info.txt'
    if i not in miss_course:
        with codecs.open(fname,encoding="utf8") as f:
            content = f.readlines()
        title.append(content[1].replace('\n',''))
        enrollment.append(content[3].replace('\n',''))
        rating.append(content[4].replace('\n',''))
        num_ratings.append(content[5].replace('\n',''))
        description.append(content[7].replace('\n','') + " "+content[8].replace('\n','')+ " "+content[9].replace('\n',''))
        five_stars.append(content[10].replace('\n',''))
        four_stars.append(content[11].replace('\n',''))
        three_stars.append(content[12].replace('\n',''))
        two_stars.append(content[13].replace('\n',''))
        try:
            downloadable_resources.append(re.findall(r'\d+', content[17])[0])
        except:
            downloadable_resources.append(0)
        f.close()

In [7]:
def convert_star(d):
    """
    Extract percentage value from a string.
    :param d: A string have percentage.
    :return: A percentage value between 0 and 1.
    """
    tmp = re.findall(r'\d+', d)
    if len(tmp)>0:
        return float(tmp[0])/100
    else:
        return 0

In [8]:
data['enrollment'] = pd.Series(enrollment).replace("NULL","0").replace("","0").astype(int)
data['rating'] = pd.Series(rating).replace("NULL","0").astype(float)
data['num_ratings'] = pd.Series(num_ratings).replace("NULL","0").astype(float)
data['five_stars'] = pd.Series(five_stars).apply(convert_star)
data['four_stars'] = pd.Series(four_stars).apply(convert_star)
data['three_stars'] = pd.Series(three_stars).apply(convert_star)
data['two_stars'] = pd.Series(two_stars).apply(convert_star)
data['description'] = pd.Series(description)
data['downloadable_resources'] = pd.Series(downloadable_resources).astype(int)
data['description'] = pd.Series(description)


### Parse intructor information.

In [9]:
instructor_no_courses = []
instructor_info = []

In [10]:
def convert_number(d):
    """
    Extract int value from a string.
    :param d: A string with integer.
    :return: The integer value.
    """
    d = d.replace(",", "")
    tmp = re.findall(r'\d+', d)
    if len(tmp)>0:
        return tmp[0]
    else:
        return 0

In [11]:
for i in range(total_num):
    fname = '../data/raw/'+str(i+1)+'_instructor.txt'
    if i not in miss_course:
        with codecs.open(fname,encoding="utf8") as f:
            content = f.readlines()
        instructor_no_courses.append(convert_number(content[3]))
        instructor_info.append(content[6].replace('\n',''))

In [12]:
data['instructor_info'] = pd.Series(instructor_info)
data['instructor_no_courses'] = pd.Series(instructor_no_courses).astype(int)

In [13]:
data = data.drop([1009,1010,1011,1012])
data.head()

,id,name,seller,price,original_price,lectures,hours,level,enrollment,rating,num_ratings,five_stars,four_stars,three_stars,two_stars,description,downloadable_resources,instructor_info,instructor_no_courses
0,1,the-complete-sql-bootcamp/,Jose Portilla,25.99,179.99,83,9.0,All Levels,342288.0,4.7,91772.0,0.60,0.33,0.06,0.01,Use SQL to query a database Use SQL to perform...,14.0,Jose Marcial Portilla has a BS and MS in Mec...,31.0
1,2,tableau10/,Kirill Eremenko SuperDataScience Team,16.99,124.99,82,9.0,All Levels,231631.0,4.6,61533.0,0.60,0.33,0.06,0.01,Install Tableau Desktop 2020 Connect Tableau t...,6.0,"Hi there, We are the Ligency PR and Marketing ...",48.0
2,3,powerbi-complete-introduction/,Manuel Lorenz Academind by Maximilian Schwarz...,12.99,94.99,275,23.5,Beginner,148077.0,4.6,43339.0,0.54,0.38,0.07,0.01,At the end of this course students will be abl...,26.0,Experience as (Web) Developer Starting out at ...,35.0
3,4,microsoft-power-bi-up-running-with-power-bi-de...,Maven Analytics Chris Dutton Aaron Parry,19.99,139.99,117,11.0,All Levels,109303.0,4.6,30899.0,0.61,0.33,0.06,0.01,Build professional quality business intelligen...,14.0,Aaron is a professional analytics consultant a...,3.0
4,5,the-business-intelligence-analyst-course-2018/,365 Careers,12.99,94.99,413,20.5,All Levels,136869.0,4.5,28595.0,0.52,0.37,0.10,0.01,Become an expert in Statistics SQL Tableau ...,151.0,365 Careers is the #1 best-selling provider of...,68.0


## 3. Save the Clean Dataset

In [14]:
data.to_pickle("data.pkl")